## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
ydp = [37.51569886583137, 126.90768824079629]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_001/_census_data_2023_bnd_dong_bnd_dong_11190_2023_2023\\bnd_dong_11190_2023_2023_2Q.shp"
ydp_gdf = gpd.read_file(shapefile_path)
ydp_gdf = ydp_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/19.영등포_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 영등포구 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/6.영등포_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_map = KeplerGl(height=1000, width=1500)
ydp_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_map.add_data(data=grid_map_df, name="영등포구 격자 데이터")
ydp_map.add_data(data=roadsystem_df, name="영등포구 도로망 데이터")
# 맵 출력 및 상세설정
ydp_map

In [40]:
# 맵 저장
ydp_map.save_to_html(file_name="visualization/영등포구 현황/영등포구 기본정보 map.html")

Map saved to visualization/영등포구 기본정보 map.html!


#### 추정교통량 시각화

In [ ]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

traffic_df = pd.read_csv('SBJ_2406_001/5.영등포_추정교통량.csv')
traffic_df['year'] = traffic_df['year'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df = traffic_df[traffic_df['timeslot'] == 'all'] # 일별 누적교통량 사용
traffic_df['geometry'] = traffic_df['link_id'].map(road_geo_dic)
traffic_df = gpd.GeoDataFrame(traffic_df, geometry='geometry')
traffic_df

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_road_estimating_map = KeplerGl(height=1000, width=1500)
ydp_road_estimating_map.add_data(data=traffic_df, name="영등포구 추정교통량 데이터")
ydp_road_estimating_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
# 맵 출력 및 상세설정
ydp_road_estimating_map

In [35]:
# 맵 저장
ydp_road_estimating_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 추정교통량 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 추정교통량 map.html!


#### 도로명주소(도로)

In [65]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/7.영등포_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_df = pd.json_normalize(geojson_data['features'])
roadname_df['geometry.coordinates'] = roadname_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadname_df.rename(columns={'geometry.coordinates' : 'geometry'}, inplace=True)
# roadname_df 데이터프레임을 GeoDataFrame으로 변환
roadname_df = gpd.GeoDataFrame(roadname_df, geometry='geometry')

#### 교통사고이력 시각화

In [57]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/8.영등포_교통사고이력.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
accident_df = pd.json_normalize(geojson_data['features'])
accident_df['geometry'] = accident_df['geometry.coordinates'].apply(lambda x : make_pol(x))

# accident_df 데이터프레임을 GeoDataFrame으로 변환
accident_df = gpd.GeoDataFrame(accident_df, geometry='geometry')
accident_df.columns = [i.split("_")[-1] for i in accident_df.columns]
# 교통사고 건수 통합(3개년 통합)
accident_df['tot_acc'] = accident_df['2020'] + accident_df['2021'] + accident_df['2022']
accident_df['tot_acc'] = accident_df['tot_acc'].fillna(0)
accident_df.rename(columns={'properties.gid' : 'gid'}, inplace=True)
accident_df = accident_df[['gid', 'tot_acc', 'geometry']]
accident_df = accident_df[accident_df['tot_acc'] != 0]

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_accident_map = KeplerGl(height=1000, width=1500)
ydp_accident_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_accident_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")
ydp_accident_map.add_data(data=accident_df, name="영등포구 교통사고 데이터")

# 맵 출력 및 상세설정
ydp_accident_map

In [60]:
# 맵 저장
ydp_accident_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 교통사고이력 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 교통사고이력 map.html!


#### 불법주정차 단속이력 시각화

In [71]:
illegal_parking_df = pd.read_csv('SBJ_2406_001/24.영등포_불법주정차단속이력.csv')
illegal_parking_df['cd_ymd'] = pd.to_datetime(illegal_parking_df['cd_ymd'], format='%Y%m%d')
illegal_parking_df = geo_transform(illegal_parking_df)
illegal_parking_df

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



,cd_ymd,lon,lat,geometry
0,2021-01-01,126.891659,37.532106,POINT (126.89166 37.53211)
1,2021-01-01,126.892012,37.531427,POINT (126.89201 37.53143)
2,2021-01-01,126.902804,37.500968,POINT (126.90280 37.50097)
3,2021-01-01,126.932557,37.523259,POINT (126.93256 37.52326)
4,2021-01-01,126.922281,37.505354,POINT (126.92228 37.50535)
...,...,...,...,...
549810,2023-12-31,126.926353,37.524632,POINT (126.92635 37.52463)
549811,2023-12-31,126.920194,37.530227,POINT (126.92019 37.53023)
549812,2023-12-31,126.900990,37.490421,POINT (126.90099 37.49042)
549813,2023-12-31,126.902166,37.516641,POINT (126.90217 37.51664)


In [ ]:
# 공간 조인 수행
joined = gpd.sjoin(illegal_parking_df, grid_map_df, how="inner", op='within')
# 각 격자 내의 불법 주정차 점 개수 계산
grid_parking_counts = joined.groupby(joined.index_right).size()
# 결과를 grid_map_df에 추가
grid_map_df_illegal = grid_map_df.copy()
grid_map_df_illegal['illegal_parking_count'] = grid_parking_counts
# NaN 값을 0으로 대체
grid_map_df_illegal['illegal_parking_count'] = grid_map_df_illegal['illegal_parking_count'].fillna(0)

In [ ]:
# 맵 객체 생성 및 데이터 로드
ydp_illegal_parking_map = KeplerGl(height=1000, width=1500)
ydp_illegal_parking_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
ydp_illegal_parking_map.add_data(data=roadname_df, name="영등포구 도로망 데이터")
ydp_illegal_parking_map.add_data(data=illegal_parking_df, name="영등포구 불법주정차단속 데이터")
ydp_illegal_parking_map.add_data(data=grid_map_df_illegal, name="영등포구 불법주정차단속 격자매핑 데이터")

# 맵 출력 및 상세설정
ydp_illegal_parking_map

In [80]:
# 맵 저장
ydp_illegal_parking_map.save_to_html(file_name="visualization/영등포구 교통관련/영등포구 불법주정차단속이력 map.html")

Map saved to visualization/영등포구 교통관련/영등포구 불법주정차단속이력 map.html!
